In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os
from dotenv import load_dotenv,find_dotenv
import pandas as pd

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableSequence

In [11]:
_=load_dotenv(find_dotenv())

In [12]:
llm_model="gemini-1.5-flash"

In [13]:
df=pd.read_csv('data.csv')

In [14]:
llm=ChatGoogleGenerativeAI(tempurature=0.9,model=llm_model)

In [15]:
first_prompt=ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}"
)

In [16]:
second_prompt=ChatPromptTemplate.from_template(
    "Write a 20-word description for the following compant: {company_name}"
)

In [17]:
chain_one=first_prompt | llm
chain_two=second_prompt | llm

In [18]:
overall_simple_chain=RunnableSequence(chain_one | chain_two)

In [19]:
product="Queen Size Sheet Set"

In [20]:
response=overall_simple_chain.invoke({"product":product})

In [21]:
print(response)

content='Luxury bedding brand offering superior comfort and quality.  Names evoke serenity, relaxation, and regal elegance.  Perfect for discerning customers.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-a4783d06-0678-4dbe-b85d-33b3ddfadc7b-0' usage_metadata={'input_tokens': 471, 'output_tokens': 27, 'total_tokens': 498, 'input_token_details': {'cache_read': 0}}


In [26]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import SequentialChain,LLMChain

In [27]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [28]:
first_prompt=ChatPromptTemplate.from_template(
    "Translate the following review to English:\n\n{Review}"
)

In [29]:
chain_one=LLMChain(llm=llm,prompt=first_prompt,output_key="English_Review")

In [30]:
second_prompt=ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:\n\n{English_Review}"
)

In [31]:
chain_two=LLMChain(llm=llm,prompt=second_prompt,output_key="summary")

In [32]:
third_prompt=ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

In [34]:
chain_three=LLMChain(llm=llm,prompt=third_prompt,output_key="language")

In [35]:
fourth_prompt=ChatPromptTemplate.from_template(
    "Write a follow-up response to the folllowing summary in the specified language:\n\Summary: {summary}\n\nLanguage: {language}"
)

In [36]:
chain_four=LLMChain(llm=llm,prompt=fourth_prompt,output_key="followup_message")

In [41]:
overall_chain=SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four],
    input_variables=["Review"],
    output_variables=["English_Review","summary","followup_message"],
    verbose=True
)

In [42]:
df=pd.DataFrame({"Review":["Esta es una reseña de ejemplo.", "Ceci est un avis d'exemple."]})

In [43]:
review=df.Review[0]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Esta es una reseña de ejemplo.',
 'English_Review': 'This is an example review.',
 'summary': 'This is a placeholder review with no content to summarize.',
 'followup_message': 'Pues entonces, ¡no hay nada que comentar!  No hay resumen que seguir.  ¿Deberíamos buscar una reseña real para poder discutirla?'}